In [7]:
import csv
from collections import defaultdict

DATA_FOLDER = "data/L_10_25_2/Proposed/"

transmit_count_dict = defaultdict(int)
rreq_count_dict = defaultdict(int)
rrep_count_dict = defaultdict(int)

# Transmit Count 읽기
with open(f"{DATA_FOLDER}transmitCount.txt", "r") as file:
    for line in file:
        parts = line.split(":")
        if len(parts) == 2:
            node = parts[0].strip()
            transmit_count_dict[node] += 1

total_transmit_count = sum(transmit_count_dict.values())

node_fail_count = defaultdict(int)
rsu_fail_count = 0

# Failed Count 읽기
with open(f"{DATA_FOLDER}failedCount.txt", "r") as file:
    failed_lines = file.readlines()

for line in failed_lines:
    parts = line.split(":")
    if len(parts) < 2:
        continue
    
    key, _ = parts[0].strip().split("[")
    number = parts[0].strip().split("[")[1].strip("]")

    if key == "node":
        node_fail_count[number] += 1
    elif key == "rsu":
        rsu_fail_count += 1

total_failed_count = len(failed_lines)

node_fail_ratios = {}
for node, fail_count in node_fail_count.items():
    total_transmissions = transmit_count_dict.get(f"node[{node}]", 0)
    if total_transmissions > 0:
        node_fail_ratios[node] = (fail_count / total_transmissions) * 100
    else:
        node_fail_ratios[node] = 0

average_node_fail_ratio = sum(node_fail_ratios.values()) / len(node_fail_ratios) if node_fail_ratios else 0
rsu_fail_ratio = (rsu_fail_count / total_transmit_count) * 100 if total_transmit_count > 0 else 0
overall_fail_ratio = (total_failed_count / total_transmit_count) * 100 if total_transmit_count > 0 else 0

# RREQ Count 읽기
with open(f"{DATA_FOLDER}rreqCount.txt", "r") as file:
    for line in file:
        node = line.strip()
        rreq_count_dict[node] += 1

# RREP Count 읽기
with open(f"{DATA_FOLDER}rrepCnt.txt", "r") as file:
    for line in file:
        node = line.strip()
        rrep_count_dict[node] += 1

# CSV 저장
csv_filename = "failure_analysis.csv"
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    
    writer.writerow(["Node", "Transmit Count", "Fail Count", "Failure Rate (%)", "RREQ Count", "RREP Count"])
    
    for node in transmit_count_dict.keys():
        fail_count = node_fail_count.get(node.split("[")[1].strip("]"), 0)
        failure_rate = node_fail_ratios.get(node.split("[")[1].strip("]"), 0)
        rreq_count = rreq_count_dict.get(node, 0)
        rrep_count = rrep_count_dict.get(node, 0)
        writer.writerow([node, transmit_count_dict[node], fail_count, failure_rate, rreq_count, rrep_count])
    
    writer.writerow([])
    writer.writerow(["Total Transmit Count", total_transmit_count])
    writer.writerow(["Total Failed Count", total_failed_count])
    writer.writerow(["Average Node Failure Rate (%)", average_node_fail_ratio])
    writer.writerow(["RSU Failure Rate (%)", rsu_fail_ratio])
    writer.writerow(["Overall Failure Rate (%)", overall_fail_ratio])

print(f"CSV 파일 저장 완료: {csv_filename}")


CSV 파일 저장 완료: failure_analysis.csv
